In [11]:
import tensorflow as tf
import numpy as np
import random
from sklearn.datasets import fetch_california_housing
from IPython.display import clear_output, Image, display, HTML

###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
###### Do not modify  here ######

###### Implement Data Preprocess here ######
housing = fetch_california_housing()
print("Shape of dataset:", housing.data.shape)
print("Shape of label:", housing.target.shape)
print(housing.DESCR)
print("Features:", housing.feature_names)


###### Implement Data Preprocess here ######

Shape of dataset: (20640, 8)
Shape of label: (20640,)
California housing dataset.

The original database is available from StatLib

    http://lib.stat.cmu.edu/

The data contains 20,640 observations on 9 variables.

This dataset contains the average house value as target variable
and the following input variables (features): average income,
housing average age, average rooms, average bedrooms, population,
average occupation, latitude, and longitude in that order.

References
----------

Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,
Statistics and Probability Letters, 33 (1997) 291-297.


Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [15]:
def normalization(dataset):
    mean = dataset.mean()
    std = dataset.std()
    normalized_dataset = (dataset - mean) / std
    return normalized_dataset

In [26]:
#Prepare train and test data
split_point = housing.data.shape[0]*9//10
X_train, X_test = housing.data[:split_point], housing.data[split_point:]
y_train, y_test = housing.target[:split_point], housing.target[:split_point]

In [14]:
X_train.data.shape

(18576, 8)

In [29]:
#Input of TF model
x = tf.placeholder(tf.float32, [None, X_train.shape[1]])
y = tf.placeholder(tf.float32, [None, 1])

#Y = W * X + b, Define W for each input
W = tf.Variable(tf.random_uniform([X_train.shape[1], 1], -1.0, 1.0, tf.float32), name = "weights")

#Define bias(b) for linear function
b = tf.Variable(tf.zeros([1]), name = "bias")

#Construct the model
y_ = tf.matmul(x, W) + b

#The loss(cost) function. Minimize the mean squared errors.
loss = tf.reduce_mean(tf.square(y_ - y))

#Using gradien descent as optimizer to minimize the coss.
optimizer = tf.train.GradientDescentOptimizer(0.05)
train = optimizer.minimize(loss)

In [28]:
###### Start TF session ######
with tf.Session() as sess:
    show_graph(tf.get_default_graph().as_graph_def())
###### Start TF session ######